# Initialize

In [1]:
import requests
import mysql.connector
from mysql.connector import Error
import os

In [ ]:
#API HEALTH CHECK: connecting to api - internal port 80 (from docker)
response = requests.get("http://api:80/health")
print(response.json())

In [ ]:
# Columns from your dataset
columns = [
    "Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways",
    "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points", "Wilderness_Area",
    "Soil_Type", "Cover_Type"
]

In [ ]:
# The FastAPI service is named 'api' in docker-compose and runs on port 80
url = "http://api:80/data"
params = {"group_number": 10}

response = requests.get(url, params=params)
print(response.json())

# Database Functions

## Create cursor

In [2]:
def get_mysql_cursor():
    """
    Establishes a MySQL connection using environment variables and returns the cursor and connection.
    """
    conn = None
    try:
        host = os.getenv("MYSQL_HOST")
        port = int(os.getenv("MYSQL_PORT"))
        user = os.getenv("MYSQL_USER")
        password = os.getenv("MYSQL_PASSWORD")
        database = os.getenv("MYSQL_DATABASE")
    
        conn = mysql.connector.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=database
        )

        if conn.is_connected():
            print("MySQL cursor created successfully")
            return conn.cursor(), conn
        else:
            print("Failed to connect to MySQL.")
            return None, None

    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
        return None, None


In [3]:
get_mysql_cursor()

MySQL cursor created successfully


(<mysql.connector.cursor_cext.CMySQLCursor at 0x7fb8a051f010>,
 <mysql.connector.connection_cext.CMySQLConnection at 0x7fb8a051ef90>)

## Get DBs and Tables

In [4]:
def list_mysql_databases_and_tables():
    """
    Lists all databases and their tables in the connected MySQL server.
    """
    cursor, conn = get_mysql_cursor()
    try:
        cursor.execute("SHOW DATABASES;")
        databases = cursor.fetchall()
        print("Databases and their tables:")

        for (db_name,) in databases:
            print(f"\n📁 Database: {db_name}")
            try:
                cursor.execute(f"USE `{db_name}`;")
                cursor.execute("SHOW TABLES;")
                tables = cursor.fetchall()
                if tables:
                    for (table_name,) in tables:
                        print(f"  - 🗂️ {table_name}")
                else:
                    print("  (No tables found)")
            except Error as e:
                print(f"  ⚠️ Could not access tables in '{db_name}': {e}")
    finally:
        cursor.close()
        conn.close()


In [5]:
list_mysql_databases_and_tables()

MySQL cursor created successfully
Databases and their tables:

📁 Database: covertype
  - 🗂️ covertype_data

📁 Database: information_schema
  - 🗂️ ADMINISTRABLE_ROLE_AUTHORIZATIONS
  - 🗂️ APPLICABLE_ROLES
  - 🗂️ CHARACTER_SETS
  - 🗂️ CHECK_CONSTRAINTS
  - 🗂️ COLLATIONS
  - 🗂️ COLLATION_CHARACTER_SET_APPLICABILITY
  - 🗂️ COLUMNS
  - 🗂️ COLUMNS_EXTENSIONS
  - 🗂️ COLUMN_PRIVILEGES
  - 🗂️ COLUMN_STATISTICS
  - 🗂️ ENABLED_ROLES
  - 🗂️ ENGINES
  - 🗂️ EVENTS
  - 🗂️ FILES
  - 🗂️ INNODB_BUFFER_PAGE
  - 🗂️ INNODB_BUFFER_PAGE_LRU
  - 🗂️ INNODB_BUFFER_POOL_STATS
  - 🗂️ INNODB_CACHED_INDEXES
  - 🗂️ INNODB_CMP
  - 🗂️ INNODB_CMPMEM
  - 🗂️ INNODB_CMPMEM_RESET
  - 🗂️ INNODB_CMP_PER_INDEX
  - 🗂️ INNODB_CMP_PER_INDEX_RESET
  - 🗂️ INNODB_CMP_RESET
  - 🗂️ INNODB_COLUMNS
  - 🗂️ INNODB_DATAFILES
  - 🗂️ INNODB_FIELDS
  - 🗂️ INNODB_FOREIGN
  - 🗂️ INNODB_FOREIGN_COLS
  - 🗂️ INNODB_FT_BEING_DELETED
  - 🗂️ INNODB_FT_CONFIG
  - 🗂️ INNODB_FT_DEFAULT_STOPWORD
  - 🗂️ INNODB_FT_DELETED
  - 🗂️ INNODB_FT_INDEX_CACHE
  - 

## Create Table

In [ ]:
def create_covertype_table(table_name):
    """
    Creates a table with the specified name in the MySQL database if it does not exist.
    Credentials and connection info are read from environment variables.
    
    Parameters:
    - table_name (str): Name of the table to be created.
    """

    cursor, conn = get_mysql_cursor()
    
    create_table_sql = f"""
    CREATE TABLE IF NOT EXISTS `{table_name}` (
        id INT AUTO_INCREMENT PRIMARY KEY,
        Elevation INT,
        Aspect INT,
        Slope INT,
        Horizontal_Distance_To_Hydrology INT,
        Vertical_Distance_To_Hydrology INT,
        Horizontal_Distance_To_Roadways INT,
        Hillshade_9am INT,
        Hillshade_Noon INT,
        Hillshade_3pm INT,
        Horizontal_Distance_To_Fire_Points INT,
        Wilderness_Area VARCHAR(20),
        Soil_Type VARCHAR(20),
        Cover_Type INT
    );
    """
    try:
        cursor.execute(create_table_sql)
        conn.commit()
        print(f"Table '{table_name}' created or already exists.")
    
    finally:
        cursor.close()
        conn.close()


In [ ]:
# Example usage:
table_name = 'covertype_raw_data'
create_covertype_table(table_name)

## Delete table

In [ ]:
def delete_covertype_table(table_name):
    """
    Deletes the covertype_data table from the specified MySQL database if it exists.
    Credentials and connection info are read from environment variables.
    Parameters:
    - table_name (str): Name of the table to be DELETED.
    """
    cursor, conn = get_mysql_cursor()

    delete_table_sql = f"""DROP TABLE IF EXISTS `{table_name}`;"""

    try:
        cursor.execute(delete_table_sql)
        conn.commit()
        print(f"Table '{table_name}' deleted if it existed.")
    
    finally:
        cursor.close()
        conn.close()


In [ ]:
delete_covertype_table(table_name)

## Get data - Insert data

In [ ]:
def fetch_and_insert_covertype_data(group_number=1):
    # Fetch data from API
    url = "http://api:80/data"
    params = {"group_number": group_number}
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()["data"]  # Adjust if your API response structure is different

    cursor, conn = get_mysql_cursor()

    # Insert data into covertype_data table
    insert_sql = """
        INSERT INTO covertype_data (
            Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology,
            Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways,
            Hillshade_9am, Hillshade_Noon, Hillshade_3pm,
            Horizontal_Distance_To_Fire_Points, Wilderness_Area,
            Soil_Type, Cover_Type
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
    """
    try:
        for row in data:
            cursor.execute(insert_sql, tuple(row))
        conn.commit()
        print(f"Inserted {len(data)} rows into covertype_data.")
    finally:
        cursor.close()
        conn.close()


In [ ]:
# Example usage:
fetch_and_insert_covertype_data(group_number=1)

## Read Table - fecth 5 random registers

In [ ]:
def fetch_random_5_covertype_records():
    """
    Fetches 5 random records from the covertype_data table.
    Returns the result as a list of tuples.
    """
    cursor, conn = get_mysql_cursor()

    select_sql = """
        SELECT * FROM covertype_data
        ORDER BY RAND()
        LIMIT 5;
    """

    try:
        cursor.execute(select_sql)
        results = cursor.fetchall()
        for row in results:
            print(row)
        return results
    finally:
        cursor.close()
        conn.close()


In [ ]:
fetch_random_5_covertype_records()

## Read Table - get all data

In [ ]:
def fetch_all_covertype_records(table_name):
    """
    Fetches all records from the covertype_data table and prints the total number of records.
    Returns the result as a list of tuples.
    Parameters:
    - table_name (str): Name of the table.
    """
    cursor, conn = get_mysql_cursor()

    select_sql = f"""SELECT * FROM `{table_name}`;"""
    

    try:
        cursor.execute(select_sql)
        results = cursor.fetchall()
        print(f"Total records in '{table_name}': {len(results)}")
        #for row in results:
        #    print(row)
        return results
    finally:
        cursor.close()
        conn.close()


In [ ]:
fetch_all_covertype_records(table_name)

# API Functions

## Get batch from API

In [ ]:
def get_data_from_api(group_number):
    """
    Retrieves data from the API for the given group number.
    Raises an exception if the API indicates that there is no more data.
    """
    url = "http://api:80/data"
    params = {"group_number": group_number}

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json().get("data", [])
        print(f"Number of records retrieved on this batch: {len(data)}")
        if not data:
            raise ValueError("No more data available from the API.")

        return data

    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        raise

    except ValueError as ve:
        print(f"API response error: {ve}")
        raise


In [ ]:
group_number = 1 #there are 10 groups in total
batch_data = get_data_from_api(group_number)

## Insert batch into Table (with data validation - duplicates)

In [ ]:
def insert_unique_covertype_data(data, table_name):
    """
    Inserts only new records into the specified table.
    Checks for duplicates before inserting.

    Parameters:
    - data (list of tuples): Rows to insert.
    - table_name (str): Name of the table to insert data into.
    """
    cursor, conn = get_mysql_cursor()

    # Use parameterized table name safely
    check_sql = f"""
        SELECT COUNT(*) FROM `{table_name}`
        WHERE Elevation = %s AND Aspect = %s AND Slope = %s AND
              Horizontal_Distance_To_Hydrology = %s AND Vertical_Distance_To_Hydrology = %s AND
              Horizontal_Distance_To_Roadways = %s AND Hillshade_9am = %s AND
              Hillshade_Noon = %s AND Hillshade_3pm = %s AND
              Horizontal_Distance_To_Fire_Points = %s AND Wilderness_Area = %s AND
              Soil_Type = %s AND Cover_Type = %s
    """

    insert_sql = f"""
        INSERT INTO `{table_name}` (
            Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology,
            Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways,
            Hillshade_9am, Hillshade_Noon, Hillshade_3pm,
            Horizontal_Distance_To_Fire_Points, Wilderness_Area,
            Soil_Type, Cover_Type
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
    """

    inserted_count = 0

    try:
        for row in data:
            cursor.execute(check_sql, tuple(row))
            exists = cursor.fetchone()[0]

            if exists == 0:
                cursor.execute(insert_sql, tuple(row))
                inserted_count += 1

        conn.commit()
        print(f"Inserted {inserted_count} new rows into '{table_name}'.")
    finally:
        cursor.close()
        conn.close()


In [ ]:
insert_unique_covertype_data(batch_data, table_name)

In [ ]:
results = fetch_all_covertype_records(table_name)

In [6]:
#Convert this notebook to .py
#pip show nbconvert

In [ ]:
!jupyter nbconvert --to script 02_data_functions.ipynb